In [42]:
import numpy as np
import pandas as pd
import os
import pickle
import math
import pdb
from matplotlib import pyplot as plt
import importlib
from scipy.optimize import fsolve

import loaders
import geography
import visualization

importlib.reload(loaders)
importlib.reload(geography)
importlib.reload(visualization)
import visualization as vis

%matplotlib

Using matplotlib backend: Qt5Agg


In [49]:
specs = loaders.load_density()
specs = geography.get_coordinate_functions(specs)
airports = loaders.load_airports(specs)

travel = loaders.load_travel(airports)
travel = geography.augment_travel(travel['annual'], airports)
#travel = {time: augment_travel(df, airports) for time, df in travel.items()}

In [48]:
from functools import partial
kappa = 1
n_infected = 1
P_no_outbreak = lambda R: 1 - fsolve(partial(loaders.f, kappa=kappa, R=R), x0=np.array(0.99))[0]
R = 4
1/R**2, P(R)

TypeError: <lambda>() got an unexpected keyword argument 'n_infected'

In [86]:
from cartopy import crs as ccrs
from cartopy import feature as cfeature
cm = plt.cm.plasma
cm = plt.cm.coolwarm
df = travel
plt.close('all')
ax = plt.axes(projection=ccrs.PlateCarree())
tt = ax.scatter(df.origin_lon, 
           df.origin_lat,
           color = cm(df.red.values))
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.STATES)
ax.set_xticklabels([])
ax.set_yticklabels([])

sm = plt.cm.ScalarMappable(cmap=cm)
sm._A = []
cb = plt.colorbar(sm, orientation='horizontal')
#cb.set_ticks([])

#fig.colorbar(cm, ax=ax)